In [ ]:
#import pandas, matplotlib, numpy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#Create arrays of lats and lngs, declare arrays as variables
lats = np.random.uniform(low = -90.000, high = 90.000, size = 2000)
lngs = np.random.uniform(low = -180.000, high = 180.000, size = 2000)
#use zip function to create pairs of lats and lngs
lat_lngs = zip(lats, lngs)
lat_lngs

In [ ]:
#create list to hold lat_lng pairs
coordinates = list(lat_lngs)


In [ ]:
#import citipy module
from citipy import citipy

In [ ]:
#create a list to hold the city names
cities = []
#identify the nearest city for each lat_lng pair
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #if the city is unique, add the city name to the cities list
    if city not in cities:
        cities.append(city)
        
#print length of city list to ensure sufficient count
len(cities)

In [ ]:
#import requests library
import requests

from config import weather_api_key

In [ ]:
#build basic URL for the OWM with weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [ ]:
#create an empty list to hold the weather data
weather_data = []
#print the beginning of the logging
print('Beginning data retrieval    ')
print('----------------------------')

#create a record count for cities 1-50
record_count = 1
#create a set count for groups of cities
set_count = 1

#loop through cities list 
for i, city in enumerate(cities):
    
    #group cities in lists of 50
    if (i % 50 ==0 and i>=50):
        set_count  +=1
        record_count = 1
    #create endpoint URL with each city
    city_url = url + '&q=' +city.replace(' ', '+')
    
    #log the UR, record number, set number, and city
    print(f'Processing Record {record_count} of Set {set_count} | {city}')
    #add 1 to the record count
    record_count +=1
    
    #Run an API request to get info for each city
    try:
        #Parse the JSON and retrieve data
        city_data = requests.get(city_url).json()
        #parse out the needed data
        city_lat = city_data['coord']['lat']
        city_lng = city_data['coord']['lon']
        city_max_temp = city_data['main']['temp_max']
        city_humidity = city_data['main']['humidity']
        city_clouds = city_data['clouds']['all']
        city_wind = city_data['wind']['speed']
        city_country = city_data['sys']['country']
        city_weather =city_data['weather'][0]['description']
        #append the information to the weather_data list
        weather_data.append({'City': city.title(),
                            "Country": city_country,
                         'Lat': city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_humidity,
                         "Cloudiness": city_clouds,
                         "Wind Speed": city_wind,
                         "Current Description": city_weather})
    except:
        print('City not found.  Skipping...')
        pass
    
#indicate that Data Loading is complete.
print("-----------------------------")
print('Data Retrieval Complete      ')
print('-----------------------------')  

In [ ]:
#check number of cities in array
len(weather_data)

In [ ]:
#save list to data frame
weather_data_df = pd.DataFrame(weather_data)
weather_data_df.head(10)

In [ ]:
#create output file for CSV
output_data_file = 'WeatherPy_Database.csv'
#export to CSV
weather_data_df.to_csv(output_data_file, index_label = 'City_ID')